In [1]:
import pandas as pd
import numpy as np
import csv

# Documentation

seasons.csv -- seasons of snl
episodes.csv -- episodes of snl   Season has_many Episodes

titles.csv -- segments of the show.  Episode has_many Titles. Some are Sketches (have a skid).
appearances.csv -- actions appearing in titles (aid, tid, capacity-Cast/Guest).  An appearance might involve an impression or a character.

characters.csv -- actors playing characters.  referenced from appearances.csv



sketches.csv -- sketch titles.  Not clear how this is connected. It is referenced in titles.csv. A sketch is a title.


Derived data:

casts.csv -- details about cast members.


In [2]:
raw_seasons = pd.read_csv('output/seasons.csv')
raw_episodes = pd.read_csv('output/episodes.csv')
raw_titles = pd.read_csv('output/titles.csv')
raw_appearances = pd.read_csv('output/appearances.csv') 
# drop charid, impid.
raw_actors = pd.read_csv('output/actors.csv')
# type seems overdetermined?

In [3]:
(
    raw_seasons
    .rename(columns={'sid':'season_id', 
                     'first_epid': 'first_episode_id',
                     'last_epid': 'last_episode_id',
                     'n_episodes': 'num_episode_in_season'})
    .to_csv('../sat_night_live_dataset/seasons.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)    
)

In [4]:
(
    raw_episodes
    .rename(columns = { 'epid': 'episode_id',
                       'sid': 'season_id',
                       'epno': 'episode_order_in_season',
                       'aired': 'aired_at' 
    })
    .reindex(columns = ['episode_id', 'aired_at', 'season_id', 'episode_order_in_season'])
    .to_csv('../sat_night_live_dataset/episodes.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)    
)

In [5]:
(
    raw_titles
    .rename(columns = { 'epid': 'episode_id',
                        'tid': 'segment_id',
                        'order': 'segment_order_in_episode'
                      }
           )
    .drop(columns = ['skid', 'sid']) # redundent
    .reindex(columns = ['segment_id', 'name', 'category', 'episode_id', 'segment_order_in_episode'])
    .to_csv('../sat_night_live_dataset/segments.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)    
)

In [13]:
# actors needed below to change actor_name to actor_id.
actors = \
(
    raw_actors
    .rename(columns = { 'aid': 'actor_name'})
    .drop(columns = ['url'])
    .assign(actor_id = lambda df_: np.arange(df_.shape[0]) + 1) # new primary key for this table
    .reindex(columns = ['actor_id', 'actor_name', 'type', 'gender'])
)
actors.to_csv('../sat_night_live_dataset/actors.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

In [14]:
(
    raw_appearances
    .rename(columns = { 'tid': 'segment_id',
                        'aid': 'actor_name',
                        'voice': 'is_voice_role'
                      }
           )
    .drop(columns = ['sid', 'charid', 'impid', 'epid']) # redundant or too complicated
    .assign(appearance_id = lambda df_: np.arange(df_.shape[0]) + 1)  # new primary key for this table
    # make the relationship to actors go via actor_id and not actor_name
    .merge(actors, how = "left", on = 'actor_name')
    .drop(columns = ['type', 'gender', 'actor_name'])
    .reindex(columns = ['appearance_id', 'actor_id', 'role', 'is_voice_role', 'segment_id'])
    .to_csv('../sat_night_live_dataset/appearances.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)
)